In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from statsmodels.tsa.api import VAR
from sklearn.metrics import mean_squared_error, r2_score
import os
from dotenv import load_dotenv
import yfinance as yf
import pandas as pd
import pandas_ta as ta
from scikeras.wrappers import KerasRegressor
# Load environment variables from the .env file
load_dotenv('../.env');

# Data Viz. 
import statsmodels.formula.api as smf
from statsmodels.tsa.seasonal import seasonal_decompose
from scipy.ndimage import gaussian_filter
from calendar import monthrange
from calendar import month_name

import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns

In [2]:
import requests
import csv
from itertools import permutations
import matplotlib.pyplot as plt

from sklearn.metrics import mean_squared_error
from math import sqrt
from statsmodels.tsa.stattools import adfuller,kpss
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.graphics.tsaplots import plot_pacf

import statsmodels.graphics.tsaplots as tsaplot
from statsmodels.tsa.holtwinters import Holt, ExponentialSmoothing, SimpleExpSmoothing

import warnings
warnings.filterwarnings('ignore')

In [9]:
def load_and_prepare(ticker, period="max"):
    df = pd.DataFrame()
    df = df.ta.ticker(ticker, period=period)
    df.ta.adjusted = "adj_close"
    df.reset_index(inplace=True)
    df.columns = df.columns.str.lower().str.replace(' ', '_')
    df.drop(['date', 'dividends', 'stock_splits'], axis=1, inplace=True)
    
    df['rsi']  = ta.rsi(df.close, length=15)
    df['emaf'] = ta.ema(df.close, length=20) #fast moving avg
    df['emam'] = ta.ema(df.close, length=100) #medium moving avg
    df['emas'] = ta.ema(df.close, length=150) #slow moving avg
    #df['macd'] = ta.macd(df.close, fast=12, slow=26, signal=9) #moving average convergance and divergance
    
    df['vma']  = ta.sma(df.volume, length=20) # Volume Moving Average
    df['vrc']  = ta.roc(df.volume, length=5) # Volume Rate of change
    
    df['target']            = df.close - df.open
    df['target']            = df.target.shift(-1)
    df['target_class']      = [1 if df.target[i]>0 else 0 for i in range(len(df))]
    df['target_next_close'] = df.close.shift(-1)

    df.dropna(inplace=True)
    return df

In [10]:
df=load_and_prepare("AAPL")

In [11]:
df

,open,high,low,close,volume,rsi,emaf,emam,emas,vma,vrc,target,target_class,target_next_close
149,0.089387,0.089818,0.089387,0.089387,19824000,45.489537,0.090299,0.097269,0.097453,31013920.0,-64.068210,-0.000432,0,0.083342
150,0.083773,0.083773,0.083342,0.083342,23654400,39.340084,0.089636,0.096994,0.097266,30821280.0,-48.285994,-0.000432,0,0.082910
151,0.083342,0.083342,0.082910,0.082910,31942400,38.937224,0.088996,0.096715,0.097076,31876320.0,61.494904,-0.000864,0,0.078159
152,0.079024,0.079024,0.078159,0.078159,22668800,34.743562,0.087964,0.096347,0.096826,32258240.0,106.952965,0.000000,0,0.080319
153,0.080319,0.080751,0.080319,0.080319,34451200,37.996474,0.087236,0.096030,0.096607,32829440.0,126.176471,0.000000,0,0.082910
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10850,192.490005,193.500000,191.089996,193.149994,48087700,54.066366,193.279597,184.169574,180.655088,53895235.0,18.110679,-0.559998,0,193.580002
10851,194.139999,194.660004,193.169998,193.580002,34049900,55.138020,193.308207,184.355919,180.826279,53447020.0,-34.823746,-1.369995,0,192.529999
10852,193.899994,194.399994,191.729996,192.529999,42628800,51.966107,193.234092,184.517782,180.981295,53138740.0,-8.290647,-1.509995,0,185.639999
10853,187.149994,188.440002,183.889999,185.639999,82488700,37.001110,192.510845,184.540004,181.043000,54979210.0,122.204952,0.029999,1,184.250000


In [ ]:
def stationarity_test(stock_close_price):
    
    # Calculate rolling mean and rolling standard deviation
    rolling_mean = stock_close_price.rolling(30).mean()
    rolling_std_dev = stock_close_price.rolling(30).std()
    
    # Plot the statistics
    plt.figure(figsize=(24,6))
    plt.plot(rolling_mean, color='#FF5A36', label='Rolling Mean')
    plt.plot(rolling_std_dev, color='#1E4485', label = 'Rolling Std Dev')
    plt.plot(stock_close_price, color='#99D04A',label='Original Time Series')
    plt.xticks([])
    plt.legend(loc='best')
    plt.title('Rolling Mean and Standard Deviation')
    
    # ADF test
    print("ADF Test:")
    adf_test = adfuller(stock_close_price,autolag='AIC')
    print('Null Hypothesis: Not Stationary')
    print('ADF Statistic: %f' % adf_test[0])
    print('p-value: %f' % adf_test[1])
    print('----'*10)
    
    # KPSS test
    print("KPSS Test:")
    kpss_test = kpss(stock_close_price, regression='c', nlags="legacy", store=False)
    print('Null Hypothesis: Stationary')
    print('KPSS Statistic: %f' % kpss_test[0])
    print('p-value: %f' % kpss_test[1])
    print('----'*10)
    
stationarity_test(df['close'])

In [ ]:
#log transformation first before d-trended

df['Log_close'] = np.log(df['close'])

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(df['Log_close'], label='Log Close', color='blue')
plt.title('Log Close Price Over Time')
plt.xlabel('Days')
plt.ylabel('Log Close Price')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
stationarity_test(df['Log_close'])

### De-trend the time series 

In [ ]:
# De-trending the time series
df['close_detrend'] = (df['close'] - df['close'].shift(30))

In [ ]:
# Detrend individual columns after feature engineering
df['open_detrended'] = (df['open'] - df['open'].shift(30))
df['high_detrend'] = (df['high'] - df['high'].shift(30))
df['low_detrend'] = (df['low'] - df['low'].shift(30))
df['rsi_detrend'] = (df['rsi'] - df['rsi'].shift(30))

# Assuming 'slow_moving_avg', 'medium_moving_avg', 'fast_moving_avg' are your added features
df['emas_detrended'] = (df['emas'] - df['emas'].shift(30))
df['emam_detrended'] = (df['emam'] - df['emam'].shift(30))
df['emaf_detrended'] = (df['emaf'] - df['emaf'].shift(30))

df['target_detrended'] = (df['target'] - df['target'].shift(30))
df['target_next_close_detrended'] = (df['target_next_close'] - df['target_next_close'].shift(30))

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(df['close_detrend'], label='Adjusted Close', color='blue')
plt.title('Adjusted Close Price Over Time')
plt.xlabel('Date')
plt.ylabel('Adjusted Close Price')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Test for stationarity after de-trending 
def stationarity_test(stock_close_price):
    
    # Calculate rolling mean and rolling standard deviation
    rolling_mean = stock_close_price.rolling(30).mean()
    rolling_std_dev = stock_close_price.rolling(30).std()
  
    # Plot the statistics
    plt.figure(figsize=(24,6))
    plt.plot(rolling_mean, label='Rolling Mean',linewidth=2.0)
    plt.plot(rolling_std_dev, label = 'Rolling Std Dev',linewidth=2.0)
    plt.plot(stock_close_price,label='De-Trended Time Series')
    plt.xticks([])
    plt.legend(loc='best')
    plt.title('Rolling Mean and Standard Deviation')
    plt.tight_layout()
    
    # ADF test
    print("ADF Test:")
    adf_test = adfuller(stock_close_price,autolag='AIC')
    print('Null Hypothesis: Not Stationary')
    print('ADF Statistic: %f' % adf_test[0])
    print('p-value: %f' % adf_test[1])
    print('----'*10)
    
    # KPSS test
    print("KPSS Test:")
    kpss_test = kpss(stock_close_price, regression='c', nlags='legacy', store=False)
    print('Null Hypothesis: Stationary')
    print('KPSS Statistic: %f' % kpss_test[0])
    print('p-value: %f' % kpss_test[1])
    print('----'*10)
    
stationarity_test(df['close_detrend'].dropna())

# Partial Autocorrelation Plot
pacf = plot_pacf(df['close_detrend'].dropna(), lags=30)

### Applying LSTM model

In [ ]:
df.columns

In [ ]:
df.isna().sum()

In [ ]:
df = df.dropna(subset=['target_detrended', 'target_next_close_detrended'])

In [ ]:
# Stock closing prices are features and date is the date time index
#, Removing some numeric features 'high_detrend', 'low_detrend'
numeric_features = df[['close_detrend', 'rsi_detrend', 'emaf_detrended', 'emam_detrended', 'emas_detrended',]]
targets = df[['target_detrended', 'target_class', 'target_next_close_detrended']]

train_size = int(len(df) * 0.8)
train_features, test_features = numeric_features[:train_size], numeric_features[train_size:]
train_targets, test_targets = targets[:train_size], targets[train_size:]


# Create a preprocessing pipeline
preprocessing_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', MinMaxScaler(feature_range=(0,1)))
])

# Apply the preprocessing pipeline to numeric features
preprocessor = ColumnTransformer(
    transformers=[
        ('num', preprocessing_pipeline, numeric_features.columns)
    ])

In [ ]:
# Fit and transform the training data
train_features_preprocessed = preprocessor.fit_transform(train_features)

# Transform the test data using the learned parameters from the training data
test_features_preprocessed = preprocessor.transform(test_features)

In [ ]:
def reshape_for_lstm(data, time_steps):
    return data.reshape((data.shape[0], time_steps, data.shape[1]))

time_steps = 1  # We can experiment with different time steps
train_features_lstm = reshape_for_lstm(train_features_preprocessed, time_steps)
test_features_lstm = reshape_for_lstm(test_features_preprocessed, time_steps)


In [ ]:
model = Sequential()
time_steps = 1
model.add(LSTM(units=50, activation='relu', input_shape=(time_steps, numeric_features.shape[1])))
model.add(Dense(units=3))  # Assuming you have 3 output columns

model.compile(optimizer='adam', loss='mean_squared_error')  # Adjust the loss function based on your problem

In [ ]:
epochs = 50
batch_size = 32

In [ ]:
model.fit(train_features_lstm, train_targets, epochs=epochs, batch_size=batch_size)

In [ ]:
test_loss = model.evaluate(test_features_lstm, test_targets)
print(f'Test Loss: {test_loss}')

In [ ]:
import matplotlib.pyplot as plt

# Assuming 'model' is your trained LSTM model
predictions = model.predict(test_features_lstm)

# Plotting
plt.figure(figsize=(25, 10))
plt.plot(test_targets.index, test_targets['target_next_close_detrended'], label='True Values', marker='o')
plt.plot(test_targets.index, predictions[:, 2], label='Predictions', marker='o')  # Assuming 'target_next_close' is the third column
plt.title('LSTM Model Predictions vs True Values')
plt.xlabel('Date')
plt.ylabel('Target Next Close')
plt.legend()
plt.show()


In [ ]:
history = model.fit(train_features_lstm, train_targets, epochs=epochs, batch_size=batch_size, validation_data=(test_features_lstm, test_targets))

# Plot training loss
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss Over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

predictions = model.predict(test_features_lstm)
mae = mean_absolute_error(test_targets['target_next_close_detrended'], predictions[:, 2])  # Assuming 'target_next_close' is the third column
mse = mean_squared_error(test_targets['target_next_close_detrended'], predictions[:, 2])
rmse = np.sqrt(mse)

print(f'MAE: {mae:.2f}')
print(f'MSE: {mse:.2f}')
print(f'RMSE: {rmse:.2f}')

In [ ]:
from sklearn.metrics import accuracy_score

# Assuming 'target_class' is the classification target column
predictions_class = (predictions[:, 1] > 0.5).astype(int)
accuracy = accuracy_score(test_targets['target_class'], predictions_class)

print(f'Directional Accuracy: {accuracy:.2%}')